In [ ]:
"""
Project: AnimalShelter Dashboard
File: ProjectTwoDashboard.ipynb
Author: Jarrale Butts
Created: 2024-02-25
Purpose: Create an interactive dashboard using JupyterDash to visualize and filter animal data from a MongoDB collection.
"""

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
from dash import callback_context
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Updated to match CRUD Python module file name and class name
from animal_shelter2 import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Updated with CRUD Python module file name and class name
username = "*****"
password = "*****"
host = '127.0.0.1'
port = 27017
database = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, host, port, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'animalLogo2.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Initial state for the filter
initial_filter = 'all'

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='logo-identifier-container', style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center'}, children=[
        html.A(href='https://www.snhu.edu', children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px', 'height': '200px'})),
        # Unique Identifier
        html.Div([
            html.H1("Dashboard Created by - Jarrale Butts", style={'font-size': '20px', 'padding-right': '15px'})
        ]),
    ]),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Dropdown filter
    html.Div(className='filterRow',
              children=[
                  dcc.Dropdown(
                      id='filter-dropdown',
                      options=[
                          {'label': 'Water Rescue', 'value': 'Water Rescue'},
                          {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                          {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'}
                      ],
                      value=initial_filter,
                      placeholder="Filter a Rescue",
                      style={'width': '280px'}
                  ),
                  html.Button(id='reset-button', n_clicks=0, children='Reset')
              ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),    # Imports Data
        # Features for the interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that the pie chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback creates interactive filter functionality
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value'),
     Input('reset-button', 'n_clicks')],
)
def update_dashboard(filter_value, n_clicks):
    # Load data from the shelter
    df = pd.DataFrame.from_records(shelter.read({}))

    # Remove unnecessary '_id' column
    df.drop(columns=['_id'], inplace=True)
    
    # Check if the reset button was clicked
    if callback_context.triggered_id == 'reset-button':
        n_clicks = None  # Reset n_clicks counter
        return df.to_dict('records')  # Reset filter

    # Filter data based on selected filter
    if filter_value == 'all':
        # Return all data if no filter is selected
        return df.to_dict('records')
    elif filter_value == 'Water Rescue':
        # Filter if "Water Rescue" is selected
        filtered_df = df[
            (df['breed'].str.lower().isin([
                'labrador retriever mix', 'chesapeake bay retriever', 'newfoundland'
            ])) &
            (df['sex_upon_outcome'] == 'Intact Female')
        ]
        return filtered_df.to_dict('records')
    elif filter_value == 'Mountain or Wilderness Rescue':
        # Filter if "Mountain or Wilderness Rescue" is selected
        filtered_df = df[
            (df['breed'].str.lower().isin([
                'german shepherd', 'alaskan malamute', 'old english sheepdog', 'siberian husky', 'rottweiler'
            ])) &
            (df['sex_upon_outcome'] == 'Intact Male')
        ]
        return filtered_df.to_dict('records')
    elif filter_value == 'Disaster Rescue or Individual Tracking':
        # Filter if "Disaster Rescue or Individual Tracking" is selected
        filtered_df = df[
            (df['breed'].str.lower().isin([
                'german shepherd', 'doberman pinscher', 'golden retriever', 'bloodhound', 'rottweiler'
            ])) &
            (df['sex_upon_outcome'] == 'Intact Male')
        ]
        return filtered_df.to_dict('records')
    else:
        return df.to_dict('records')

    # Return the filtered data (or all data if no filter is selected)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return (filtered_df.to_dict('records'), columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return None  # Handle cases where there's no data

    df = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)